## Final project: Finding a 'twin' neighborhood in Chicago area

### PART 1: Introduction/Business Problem <br>
In this final assignment,I will explore two areas: <br>
1)  Champaign, Urbana, and Savoy. <br>
2) Great Chicago area.<br>
The reason for doing the analysis is to see which neighborhood in Chicago is the most closest to my current living place. I am moving to Chicago in July so I am very interested to know the 'twin' neighborhood there.<br>

I plan to solve this problem in these three steps:<br>

1) Obtrain the zipcode of all areas/neighborhoods in Chicago and Champaign-Urbana-Savoy.<br>
2) Obtain the coordinate that correspdone to these areas/zipcodes<br>
3) Using the coordinate information to get the venue located in these areas<br>
3) Perform a unsupervised machine learning method (for example, k-mean) to group those neighorhoods<br>
4) See if Champaign-Urbana-Savoy areas are grouped with some Chicago neighborhoods.<br>
5) Explore those corresponding'twin neighorhoods', including checking their locations on the map, check the venue distribution information, etc.<br>




### PART 2:  Data 

I will be obtaining data from three resources: <br>
1) A publically available dataset provided by Opendatasoft, containing the coordinate informaiton of all zipcodes. I only obtained the Illinois data from this website: <br>
https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=IL <br>
This data is saved as 'IL_zip-code-latitude-and-longitude.csv' <br>

2) A website containing all Chicago area's zip code and the corresponding neighborhood name.<br>
https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php#:~:text=Chicago%20real%20estate%20for%20sale,and%2060642%20for%20River%20North.'  <br>

3) A home-made excel file containing 5 local zip codes (my current place) and the corresponding neighborhood (city) names, saved as: <br>
'Champaign_Urbana_Savoy_ZIP_coordinates.xlsx' <br>

#### Data obtaining, cleaning, and wrangling

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

Dataset 1 processing:

In [2]:
#read the csv data.
IL_zipcode=pd.read_csv("C:/Users/qsun9/OneDrive/Documents/Python_learning/Python_coursera_DS_course_IBM/Course_4_Capstone/IL_zip-code-latitude-and-longitude.csv") 
IL_zipcode.head() 

,Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint
60651;Chicago;IL;41.901485;-87.74055;-6;1;41.901485,-87.74055
62090;Venice;IL;38.67065;-90.17052;-6;1;38.67065,-90.17052
62420;Casey;IL;39.299447;-87.99863;-6;1;39.299447,-87.99863
62206;East Saint Louis;IL;38.564451;-90.1628;-6;1;38.564451,-90.16280
62611;Arenzville;IL;39.883346;-90.39332;-6;1;39.883346,-90.39332


In [3]:
IL_zipcode.shape

(1627, 1)

The data needs cleaning. I will only keep the zip, city, state, lattitue, and longitute.

In [4]:
IL_zipcode=IL_zipcode.reset_index()
IL_zipcode.head()

,index,Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint
0,60651;Chicago;IL;41.901485;-87.74055;-6;1;41.9...,-87.74055
1,62090;Venice;IL;38.67065;-90.17052;-6;1;38.67065,-90.17052
2,62420;Casey;IL;39.299447;-87.99863;-6;1;39.299447,-87.99863
3,62206;East Saint Louis;IL;38.564451;-90.1628;-...,-90.16280
4,62611;Arenzville;IL;39.883346;-90.39332;-6;1;3...,-90.39332


In [5]:
Zipcode=[]
City=[]
State=[]
Latitute=[]
Longitute=[]

for i in range(0, len(IL_zipcode)):
    temp_split=IL_zipcode.iloc[i,0].split(";")
    Zipcode.append(temp_split[0])
    City.append(temp_split[1])
    State.append(temp_split[2])
    Latitute.append(temp_split[3])
    Longitute.append(temp_split[4])

IL_zipcode['Zip']=Zipcode
IL_zipcode['City']=City
IL_zipcode['State']=State
IL_zipcode['Latitute']=Latitute
IL_zipcode['Longitute']=Longitute

IL_zipcode.head()

,index,Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint,Zip,City,State,Latitute,Longitute
0,60651;Chicago;IL;41.901485;-87.74055;-6;1;41.9...,-87.74055,60651,Chicago,IL,41.901485,-87.74055
1,62090;Venice;IL;38.67065;-90.17052;-6;1;38.67065,-90.17052,62090,Venice,IL,38.67065,-90.17052
2,62420;Casey;IL;39.299447;-87.99863;-6;1;39.299447,-87.99863,62420,Casey,IL,39.299447,-87.99863
3,62206;East Saint Louis;IL;38.564451;-90.1628;-...,-90.16280,62206,East Saint Louis,IL,38.564451,-90.1628
4,62611;Arenzville;IL;39.883346;-90.39332;-6;1;3...,-90.39332,62611,Arenzville,IL,39.883346,-90.39332


In [6]:
#check data types
IL_zipcode[['Latitute','Longitute']]=IL_zipcode[['Latitute','Longitute']].apply(pd.to_numeric, errors='coerce')
IL_zipcode.dtypes

index                                                                              object
Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint    float64
Zip                                                                                object
City                                                                               object
State                                                                              object
Latitute                                                                          float64
Longitute                                                                         float64
dtype: object

In [7]:
#drop unnecessary columns
IL_zipcode=IL_zipcode.drop(['index','Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint'], axis=1)

In [8]:
IL_zipcode.head()

,Zip,City,State,Latitute,Longitute
0,60651,Chicago,IL,41.901485,-87.74055
1,62090,Venice,IL,38.670650,-90.17052
2,62420,Casey,IL,39.299447,-87.99863
3,62206,East Saint Louis,IL,38.564451,-90.16280
4,62611,Arenzville,IL,39.883346,-90.39332


In [9]:
IL_zipcode.shape

(1627, 5)

In [10]:
len(IL_zipcode['Zip'].unique()) #nice, no duplicates

1627

In [11]:
len(IL_zipcode.loc[IL_zipcode['State'] =='IL', ])

#confirmed: all data is from 1627

1627

Dataset1 is clean. <br>


I now start the dataset 2: Chicago zipcode and neighborhood names

In [12]:
#dataset2: Chicago zipcode and area names
chicago_zipcode_area=pd.read_html('https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php#:~:text=Chicago%20real%20estate%20for%20sale,and%2060642%20for%20River%20North.'
)[0]
chicago_zipcode_area.columns=['Area','Zip']
chicago_zipcode_area.shape

(199, 2)

Remove the 'non-zipcode' rows.

In [13]:
#Filter by checking if the first element in the string is numeric.
chicago_zipcode_area=chicago_zipcode_area.loc[chicago_zipcode_area["Zip"].str[0].astype(str).str.isnumeric(), ].reset_index(drop=True)

In [14]:
chicago_zipcode_area.head()

,Area,Zip
0,Cathedral District,60611
1,Central Station,60605
2,Dearborn Park,60605
3,Gold Coast,"60610, 60611"
4,Loop,"60601, 60602, 60603, 60604, 60605, 60606, 6060..."


Split multiple zipcodes that describe the same neighborhoods.

In [15]:
chicago_zipcode_area=pd.concat([Series(row['Area'], row['Zip'].split(',')) 
                                for _, row in chicago_zipcode_area.iterrows()]).reset_index()

In [16]:
chicago_zipcode_area.head()

,index,0
0,60611,Cathedral District
1,60605,Central Station
2,60605,Dearborn Park
3,60610,Gold Coast
4,60611,Gold Coast


In [17]:
chicago_zipcode_area.shape

(353, 2)

In [18]:
chicago_zipcode_area.columns=['Zip','Area']
#check utf encoding.
chicago_zipcode_area.Zip.str.encode('utf-8').head()
#it previously when we split the strings, we had the 'space' in between and we did not remove them. 

0     b'60611'
1     b'60605'
2     b'60605'
3     b'60610'
4    b' 60611'
Name: Zip, dtype: object

In [19]:
#fix utf encoding.
chicago_zipcode_area.Zip = chicago_zipcode_area.Zip.str.strip()

In [20]:
chicago_zipcode_area.Zip.str.encode('utf-8').head()
#now it looks good.

0    b'60611'
1    b'60605'
2    b'60605'
3    b'60610'
4    b'60611'
Name: Zip, dtype: object

We need the data to display unique zip codes instead of duplicated zip codes in different rows.<br>
However, there are duplicated zips in the data. I will group those neighborhoods' names.

In [21]:
chicago_zipcode_area=chicago_zipcode_area.groupby('Zip')['Area'].apply(','.join).reset_index()

In [22]:
chicago_zipcode_area.head()

,Zip,Area
0,60004,"Arlington Heights,Prospect Heights"
1,60005,Mount Prospect
2,60007,"Elk Grove Village,Schaumburg"
3,60009,Elk Grove Village
4,60010,"Barrington,Barrington Hills,Deer Park,Hoffman ..."


In [23]:
chicago_zipcode_area.shape

(179, 2)

In [24]:
len(chicago_zipcode_area['Zip'].unique())

179

now, the final dataset2 has 179 rows containing 179 unique Zipcodes.

Start processing dataset 3: the data that saves Champaign-Urbana-Savoy zipcode and area names.

In [25]:
#Dataset3: Champaign-Urbana-Savoy zip code area data

CUS_zipcode_area=pd.read_excel("C:/Users/qsun9/OneDrive/Documents/Python_learning/Python_coursera_DS_course_IBM/Course_4_Capstone/Champaign_Urbana_Savoy_ZIP_coordinates.xlsx") 
CUS_zipcode_area

,Zip,Area
0,61820,Champaign
1,61821,Champaign
2,61822,Champaign
3,61801,Urbana
4,61802,Urbana
5,61874,Savoy


Combine Dataset2 chicago_zipcode_area and Dataset3 CUS_zipcode_area

In [26]:
Chi_CUS_zip_area=pd.concat([chicago_zipcode_area,CUS_zipcode_area]).reset_index(drop=True)
Chi_CUS_zip_area.shape 

(185, 2)

Now, the dataset contains 185 zipcodes and the corresponding area names

Finish of the data cleaning. <br>

Now, Overlap Chi_CUS_zip_area with the Dataset1. 

In [27]:
#the last 6 zip are not 'str' 
(Chi_CUS_zip_area.applymap(type)==str).iloc[175:,]

,Zip,Area
175,True,True
176,True,True
177,True,True
178,True,True
179,False,True
180,False,True
181,False,True
182,False,True
183,False,True
184,False,True


In [28]:
#Change it back to 'str'
Chi_CUS_zip_area.Zip = Chi_CUS_zip_area.Zip.astype(str).str.strip()

Now do the real merge!

In [29]:
Final_area_df=Chi_CUS_zip_area.join(IL_zipcode.set_index('Zip'), on='Zip')

In [30]:
#there is still NA
Final_area_df.iloc[1:30,]

,Zip,Area,City,State,Latitute,Longitute
1,60005,Mount Prospect,Arlington Heights,IL,42.069327,-87.984640
2,60007,"Elk Grove Village,Schaumburg",Elk Grove Village,IL,42.005978,-87.998470
3,60009,Elk Grove Village,Elk Grove Village,IL,41.811929,-87.687320
4,60010,"Barrington,Barrington Hills,Deer Park,Hoffman ...",Barrington,IL,42.160791,-88.152310
5,60012,Crystal Lake,Crystal Lake,IL,42.265643,-88.316640
6,60014,Crystal Lake,Crystal Lake,IL,42.226623,-88.330660
7,60015,"Bannockburn,Deerfield",Deerfield,IL,42.169325,-87.865560
8,60016,"Glenview,Des Plaines,Park Ridge",Des Plaines,IL,42.047178,-87.890580
9,60017,Des Plaines,Des Plaines,IL,42.028779,-87.894366
10,60018,"Des Plaines,Rosemont",Des Plaines,IL,42.008429,-87.892340


In [31]:
Final_area_df.shape

(185, 6)

In [32]:
len(Final_area_df['Zip'].unique())

185

In [33]:
#remove the NA value rows
Final_area_df=Final_area_df.dropna()

In [34]:
Final_area_df.shape
#several areas are not with lattitue information, etc.

(176, 6)

In [35]:
len(Final_area_df['Zip'].unique())

176

In [36]:
Final_area_df  
#176 final rows

,Zip,Area,City,State,Latitute,Longitute
0,60004,"Arlington Heights,Prospect Heights",Arlington Heights,IL,42.108428,-87.977230
1,60005,Mount Prospect,Arlington Heights,IL,42.069327,-87.984640
2,60007,"Elk Grove Village,Schaumburg",Elk Grove Village,IL,42.005978,-87.998470
3,60009,Elk Grove Village,Elk Grove Village,IL,41.811929,-87.687320
4,60010,"Barrington,Barrington Hills,Deer Park,Hoffman ...",Barrington,IL,42.160791,-88.152310
5,60012,Crystal Lake,Crystal Lake,IL,42.265643,-88.316640
6,60014,Crystal Lake,Crystal Lake,IL,42.226623,-88.330660
7,60015,"Bannockburn,Deerfield",Deerfield,IL,42.169325,-87.865560
8,60016,"Glenview,Des Plaines,Park Ridge",Des Plaines,IL,42.047178,-87.890580
9,60017,Des Plaines,Des Plaines,IL,42.028779,-87.894366


In [37]:
Final_area_df.to_csv('Final_area_df.csv', index=False)